# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Изучение данных

In [1]:
pip install pymystem3

     |████████████████████████████████| 63 kB 438 kB/s            
     |████████████████████████████████| 138 kB 2.5 MB/s            
     |████████████████████████████████| 61 kB 1.9 MB/s            
     |████████████████████████████████| 149 kB 4.8 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

     |████████████████████████████████| 9.5 MB 1.5 MB/s            
     |████████████████████████████████| 14.8 MB 180 kB/s            
     |████████████████████████████████| 503 kB 1.6 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install fsspec

     |████████████████████████████████| 133 kB 738 kB/s            
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

Installing mystem to /home/mary/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [9]:
try:
    data = pd.read_csv('data.csv')
except:
    data = pd.read_csv('C://Users/Мария/Documents/Мария/Яндекс.Практикум/Анализ_данных/портфолио/1p_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

В столбце 'days_employed' есть отрицательные значения. В столбцах 'days_employed' и 'total_income' (количественные переменные) есть пропущенные значения. Т.к. объём пропущенных значений - 10%, их нельзя удалять, нужно заменить на характерные (средние или медианные). Формат столба 'days_employed' - вещественные числа, нужно заменить на целочисленные.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [10]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков в обих столбцах одинаковое, нужно проверить все ли значения пропущены в одних и тех же строках.

In [11]:
print(data[(data['days_employed'].isna()) & (data['total_income'].isna())].count())

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64


In [12]:
de_median = data['days_employed'].median()
ti_median = data['total_income'].median()
data.fillna({'days_employed': de_median, 'total_income': ti_median}, inplace=True)

In [13]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Вывод

Значения прорущены в одних и тех же строках, возможно, были проблемы с выгрузкой. Или с формами для сбора данных, если эти данные получены в одно и то же время.
Пропущенные значения количественных переменных заменены на медианные, потому что трудовой стаж и доход могут очень сильно различаться, средние значения больше медианных.

### Замена типа данных

In [14]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


#### Вывод

Вещественный тип данных изменён на целочисленный. Для изменения типа данных используется метод astype. Он позволяет превращать одно число в другое.

### Обработка дубликатов

In [16]:
data['education'].str.lower()
data['family_status'].str.lower()
data['gender'].str.lower()
data['income_type'].str.lower()
data['purpose'].str.lower()

0                       покупка жилья
1             приобретение автомобиля
2                       покупка жилья
3          дополнительное образование
4                     сыграть свадьбу
                     ...             
21520               операции с жильем
21521            сделка с автомобилем
21522                    недвижимость
21523    на покупку своего автомобиля
21524           на покупку автомобиля
Name: purpose, Length: 21525, dtype: object

In [17]:
data.duplicated().sum()

54

In [18]:
data = data.drop_duplicates().reset_index(drop = True)

In [19]:
data.duplicated().sum()

0

#### Вывод

Дубликаты удалены. Для поиска дубликатов используется метод duplicated(), удаления дубликатов использован метод drop_duplicates() с методом reset_index(drop = True), чтобы не сохранять в отдельный столбец старые индексы. Дубликаты могли появиться из-за проблем с выгрузкой или с формой для сбора данных, если данные вводились повторно после собщения о том, что они не были сохранены, когда на самом деле они были сохранены.

### Лемматизация

In [20]:
def purpose_category(row):
    lemmas = m.lemmatize(row)
    for row in lemmas:
        for goal in ['автомобиль', 'свадьба', 'ремонт', 'образование']:
            if goal in row:
                return goal
        if 'жилье' in row:
            return 'недвижимость'
        if 'недвижимость' in row:
            return 'недвижимость'
        if 'строительство' in row:
            return 'недвижимость'

In [21]:
data['purpose_category'] = data['purpose'].apply(purpose_category)
data['purpose_category'].value_counts()

недвижимость    10207
автомобиль       4308
образование      4014
свадьба          2335
ремонт            607
Name: purpose_category, dtype: int64

#### Вывод

С помощью лемматизации выделены основные цели кредитов. Для лемматизации использован метод библиотеки pymystem3. Сначала определены все возможные леммы, потом выбраны те, которые подходят под описание целей, и написана функция для распределения по категориям. 

### Категоризация данных

In [22]:
data['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [23]:
data['children'] = data['children'].replace(-1, 1)
def children_category(children):
    if children == 0:
        return 'нет'
    if children >= 2:
        return 'много'
    return 'один'
data['children_category'] = data['children'].apply(children_category)
data['children_category'].value_counts()

нет      14107
один      4856
много     2508
Name: children_category, dtype: int64

In [24]:
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [25]:
data['debt'].value_counts()

0    19730
1     1741
Name: debt, dtype: int64

In [26]:
data.sort_values('total_income')
def income_category(income):
    if income <= 50000:
        return '50 тысяч и меньше'
    if income <= 100000:
        return '51 - 100 тысяч'
    if income <= 200000:
        return '101 тысяча - 200 тысяч'
    if income <= 300000:
        return '201 тысяча - 300 тысяч'
    if income <= 500000:
        return '301 тысяча - 500 тысяч'
    return 'больше 500 тысяч'
data['income_category'] = data['total_income'].apply(income_category)
data['income_category'].value_counts()

101 тысяча - 200 тысяч    11942
51 - 100 тысяч             4091
201 тысяча - 300 тысяч     3584
301 тысяча - 500 тысяч     1260
50 тысяч и меньше           372
больше 500 тысяч            222
Name: income_category, dtype: int64

In [27]:
data_dict = data[['debt', 'children_category', 'family_status', 'income_category', 'purpose_category']]

#### Вывод

Дети и уровень дохода разделены по категориям. Сначала подсчитано количество значений в столбце с детьми. Т.к. больше всего людей без детей и с одним ребёнком, все остальные отнесены в одну категорию (2 и больше). Есть 76 людей с 20 детьми, что, возможно ошибка. По уровню дохода выделены категории с очень маленьким и очень большим доходом, остальные разбиты на категории с шагом 100 тысяч. Цели распределены по категориям в предыдущем шаге.

## Шаг 3. Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [28]:
data_pivot = data.pivot_table(index=['children_category'], values='debt', aggfunc='sum')
data_pivot['all'] = data_dict.groupby('children_category')['debt'].count()
data_pivot['probability_debt'] = data_pivot['debt'] / data_pivot['all']
print(data_pivot.sort_values(by = 'probability_debt', ascending = False)) 

                   debt    all  probability_debt
children_category                               
много               233   2508          0.092903
один                445   4856          0.091639
нет                1063  14107          0.075353


#### Вывод

Люди без детей с большей вероятностью вернут кредит в срок, чем люди с детьми.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [29]:
data_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')
data_pivot['all'] = data_dict.groupby('family_status')['debt'].count()
data_pivot['probability_debt'] = data_pivot['debt'] / data_pivot['all']
print(data_pivot.sort_values(by = 'probability_debt', ascending = False)) 

                       debt    all  probability_debt
family_status                                       
Не женат / не замужем   274   2810          0.097509
гражданский брак        388   4163          0.093202
женат / замужем         931  12344          0.075421
в разводе                85   1195          0.071130
вдовец / вдова           63    959          0.065693


#### Вывод

Никогда не состоявшие в браке или состоящие в гражанском браке люди с большей вероятностью не вернут кредит в срок.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
data_pivot = data.pivot_table(index=['income_category'], values='debt', aggfunc='sum')
data_pivot['all'] = data_dict.groupby('income_category')['debt'].count()
data_pivot['probability_debt'] = data_pivot['debt'] / data_pivot['all']
print(data_pivot.sort_values(by = 'probability_debt', ascending = False))

                        debt    all  probability_debt
income_category                                      
101 тысяча - 200 тысяч  1029  11942          0.086166
51 - 100 тысяч           331   4091          0.080909
301 тысяча - 500 тысяч    92   1260          0.073016
201 тысяча - 300 тысяч   252   3584          0.070312
больше 500 тысяч          14    222          0.063063
50 тысяч и меньше         23    372          0.061828


#### Вывод

Люди с доходом от 100 до 200 тысяч реже возвращают кредит в срок, чем люди с другим доходом. Люди с самым маленьким и самым большим доходом чаще возвращают кредит в срок.

### Как разные цели кредита влияют на его возврат в срок?

In [31]:
data_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='sum')
data_pivot['all'] = data_dict.groupby('purpose_category')['debt'].count()
data_pivot['probability_debt'] = data_pivot['debt'] / data_pivot['all']
print(data_pivot.sort_values(by = 'probability_debt', ascending = False))

                  debt    all  probability_debt
purpose_category                               
автомобиль         403   4308          0.093547
образование        370   4014          0.092177
свадьба            186   2335          0.079657
недвижимость       747  10207          0.073185
ремонт              35    607          0.057661


#### Вывод

Люди, которые берут кредит на покупку автомобиля или обучение, с меньшей вероятностью вернут кредит в срок.

## Шаг 4. Общий вывод

- Люди без детей с большей вероятностью вернут кредит в срок, чем люди с детьми.
    
- Никогда не состоявшие в браке или состоящие в гражанском браке люди с большей вероятностью не вернут кредит в срок.
    
- Люди с дохдом от 100 до 200 тысяч реже возвращают кредит в срок, чем люди с другим доходом. Вероятность не вернуть кредит вовремя у людей с доходом от 51 до 100 тысяч чуть меньше. Возможно, людей с доходом от 51 до 200 тысяч можно было бы отнести в одну категорию. Люди с самым маленьким и самым большим доходом чаще возвращают кредит в срок.
    
- Люди, которые берут кредит на покупку автомобиля или обучение, с меньшей вероятностью вернут кредит в срок.